# Predicting beer consumption
Online class on Supervised Learning, Wednesday, 29th of October 2025

For BIP "Machine Learning for Data Science" by Marieke Bouma & Remi Thüss, Hanze

## Importing required modules

In [ ]:
# Install packages for the project
!pip install pandas numpy seaborn matplotlib scikit-learn scipy

In [ ]:
# Import the required building blocks
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from sklearn.metrics import f1_score

# Beer consumption as a classification issue
Let's try to predict whether "very little", "little", "much" or "very much" beer is drunk; A classification.

To this end, the data is processed in which the number of Litres is divided into four classes.
The limit of the classes is roughly based on the quartile distribution.

- I: under 22,000 Litres
- II: between 22,000 and 25,000 Litres
- III: between 25,000 and 29,000 Litres
- IV: More than 29,000 Litres

If all goes well, these classes are of comparable size (i.e. somewhat balanced).

## Data analysis and preprocessing

In [ ]:
# Back to the source
df = pd.read_csv('Beerconsumption.csv')
df.dropna(inplace=True)
df.Date = pd.to_datetime(df.Date)
df.Weekend = df.Weekend.astype(int)

### Making classes

In [ ]:
# Making the classes (this is one method, many exist)
df['Class1'] = np.where(df['Litres'] > 29000, 1, 0)
df['Class2'] = np.where(df['Litres'] > 25000, 1, 0)
df['Class3'] = np.where(df['Litres'] > 22000, 1, 0)
df['Class4'] = np.where(df['Litres'] > 10000, 1, 0)

df['Class'] = df.Class1 + df.Class2 + df.Class3 + df.Class4
df.head()

In [ ]:
# The auxiliary columns can leave again
df.drop(columns=['Class1', 'Class2', 'Class3', 'Class4'], inplace=True)
df.head()

In [ ]:
# Just a check of the number of Litres per class
perClass = df.groupby('Class').agg({'Litres':['count','sum','mean','std','min','median', 'max']}).round(1)
print(perClass)

plt.figure(figsize=(10,7))
sns.boxplot(x="Class", y="Litres", data=df)
plt.xticks(range(0,4),["I", "II", "III", "IV"])
plt.title("Classes")
plt.show()

### 1. Feature selection
Is the Date a useful feature for our goal? Would another be useful? How can you check?

In [ ]:
# Let's add the season, because why not
df['Season'] = df.Date.dt.month.map({1:1, 2:1, 3:2, 4:2, 5:2, 6:3, 7:3, 8:3, 9:4, 10:4, 11:4, 12:1})

In [ ]:
# Plot the correlation matrix for all features except the date
kolomdf = ['Season', 'AvgTemp', 'MinTemp', 'MaxTemp', 'Rainfall_mm', 'Weekend', 'Class']

plt.figure(figsize=(8,8))
cm = np.corrcoef(df[kolomdf].values.T)
sns.set(font_scale=1.5)
hm = sns.heatmap(cm,
                cbar=True,
                annot=True,
                square=True,
                fmt='.2f',
                annot_kws={'size': 15},
                yticklabels=kolomdf,
                xticklabels=kolomdf
                )
plt.show()

We'll make a simple model, using only MaxTemp and Weekend as features. AvgTemp and MinTemp are too strongly correlated with MaxTemp, and Rainfall and Season are only very weakly correlated with Class.

In [ ]:
# We'll use as the predictors/features/independent variables MaxTemp and Weekend,
# and the class is the value to be predicted (label/independent var)
labels = df.loc[:,'Class']
data = df.loc[:,['MaxTemp', 'Weekend']]

# Insight into the frequency distribution of the classes;
# They are roughly balanced (as intended)
labels[:].groupby(labels[:]).plot(kind="hist")
plt.show()

## 2. Baseline models for classification
Code a baseline model

In [ ]:
# splits the dataset in a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.25, random_state=0)


# give the size of the training and the test set again
print('shape of training set: ', X_train.shape)
print('shape of test set:     ', X_test.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 2.1 Choose a baseline classification model, fill in the blank
baseline_1 = LogisticRegression(max_iter=1000, random_state=42)

baseline_1.fit(X_train, y_train)

y_pred = baseline_1.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print('Accuracy:', acc)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# 2.2 Choose a different baseline classification model, fill in the blank
baseline_2 = KNeighborsClassifier(n_neighbors=5)

baseline_2.fit(X_train, y_train)

y_pred = baseline_2.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print('Accuracy:', acc)

#### Question: how do these models compare? Is there a difference in accuracy? Why do you think this is?

## 3/4. Tree-based models
Code a decision tree and random forest below!

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Fill in the blank with a decision tree model
dt_model = DecisionTreeClassifier(random_state=42)

dt_model.fit(X_train, y_train)

y_pred = dt_model.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print('Accuracy:', acc)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Fill in the blank with a random forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print('Accuracy:', acc)

#### Challenge: visualise the decision making process
There exist some cool libraries to visualise the decision making process in a tree-based model. Try find these libraries and apply them to one of your models!

In [ ]:
# Your code here

## 5. Neural networks
Complete the code for a neural network for the beer consumption classification issue!

In [ ]:
# # The neural network expects the classes 0 1 2 3 instead of 1 2 3 4.
# y_train = y_train - 1
# y_test = y_test - 1

In [ ]:
# # Build the model
# neural_net = Sequential([
#     Input(shape=(2,)),
#     Dense(64, activation='relu'),
#     Dense(32, activation='relu'),
#     Dense(4, activation='softmax')
# ])

# # Compile
# neural_net.compile(
#     optimizer='adam',
#     loss='sparse_categorical_crossentropy',
#     metrics=['accuracy', 'f1_score']
# )

# # Train
# neural_net.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)

# # Evaluate the model
# loss, accuracy, f1 = neural_net.evaluate(X_test[:], y_test)
# print(f"Test Accuracy: {accuracy:.2f}")

#### More on neural networks in a later class!

## 6. Metrics (Confusion matrix)
Below you can find some standard code to make and visualise a confusion matrix. The score that is usually used is called the F1-score. You will learn more on model evaluation in a later class, but here's a taste!

Use the "classification report" code to calculate the F1-scores of all of your models*. How well do they perform? Do the F1-score and accuracy differ?

*excluding the neural network, because it already shows the F1-score.

In [ ]:
confmat = confusion_matrix(y_true = y_test, y_pred = y_pred)

#image for confusion matrix
fig, ax = plt.subplots(figsize = (5, 5))
ax.matshow(confmat, alpha = 0.3)
for i in range(confmat.shape[0]):
    for j in range(confmat.shape[1]):
        ax.text(x=j, y=i,
                s=confmat[i, j],
                fontsize=18,
                va='center', ha='center')
ticks = labels.unique()
ticks.sort()
ax.set_xticks(np.arange(confmat.shape[0]), labels=ticks)
ax.set_yticks(np.arange(confmat.shape[1]), labels=ticks)
plt.xlabel('predicted label')
plt.ylabel('true label')
plt.show()

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
# [your code to compare the accuracy and f1-score for each model here]

[your analysis here]

## 7. A critical look at the data
We've seen that the data is somewhat balanced when it comes to our 4 classes. However, we get rather bad results for classification. Perhaps a four-class classification task is simply not very useful with such little data.

#### Binary classification
Rerun the entire pipeline* with only two classes. Consider with your group where to put the threshold for the classification. How do the performances change?

*from preprocessing and visualisation, to training the models, to measuring the model performances

In [ ]:
# [your code and/or markdown here]